# Amazon Nova Act SDK를 사용한 실시간 브라우저 tool

## 개요

이 튜토리얼에서는 Nova Act SDK를 사용하여 Amazon Bedrock Agentcore Browser tool과 상호작용하고 브라우저를 실시간으로 보는 방법을 배웁니다.


### 튜토리얼 세부사항


| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 대화형                                                                            |
| Agent 유형          | 단일                                                                              |
| Agentic Framework   | Nova Act                                                                         |
| LLM model           | Amazon Nova Act model                                                            |
| 튜토리얼 구성요소   | NovaAct을 사용하여 browser tool과 실시간 상호작용                                |
| 튜토리얼 분야       | 범용                                                                              |
| 예제 복잡도         | 쉬움                                                                              |
| 사용 SDK            | Amazon BedrockAgentCore Python SDK, Nova Act                                     |

### 튜토리얼 아키텍처

이 튜토리얼에서는 Nova Act를 browser tool과 함께 사용하고 브라우저를 실시간으로 보는 방법을 설명합니다.  

예제에서는 Nova Act agent에 자연어 지시사항을 전송하여 Bedrock Agentcore 브라우저에서 작업을 수행하고 브라우저를 실시간으로 확인합니다.

<div style="text-align:left">
    <img src="./images/browser-tool.png" width="50%"/>
</div>

### 튜토리얼 주요 기능

* browser tool 사용 및 실시간 확인 
* Nova Act를 browser tool과 함께 사용

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명. IAM role/user는 다음 권한이 필요합니다 https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/browser-onboarding.html#browser-credentials-config
* Amazon Bedrock AgentCore SDK
* Nova Act SDK 및 API key 

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## NovaAct를 Bedrock Agentcore Browser tool과 실시간 뷰로 사용하기

여기서는 Amazon DCV SDK를 통해 Bedrock Agentcore browser tool에 연결하는 헬퍼 함수를 사용합니다.




In [ ]:
%%writefile live_view_with_nova_act.py
from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct
from rich.console import Console
from rich.panel import Panel
import sys
import json
import argparse
sys.path.append("../interactive_tools")
from browser_viewer import BrowserViewerServer

console = Console()

from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print("using region", region)

def live_view_with_nova_act(prompt, starting_page, nova_act_key, region="us-west-2"):
    """Run the browser live viewer with display sizing."""
    console.print(
        Panel(
            "[bold cyan]Browser Live Viewer[/bold cyan]\n\n"
            "This demonstrates:\n"
            "• Live browser viewing with DCV\n"
            "• Configurable display sizes (not limited to 900×800)\n"
            "• Proper display layout callbacks\n\n"
            "[yellow]Note: Requires Amazon DCV SDK files[/yellow]",
            title="Browser Live Viewer",
            border_style="blue",
        )
    )

    try:
        # Bedrock AgentCore 브라우저 세션 생성
        with browser_session(region) as client:
            # WebSocket URL과 헤더 생성 (CDP 연결용)
            ws_url, headers = client.generate_ws_headers()

            # 브라우저 실시간 뷰어 서버 시작
            console.print("\n[cyan]Step 3: Starting viewer server...[/cyan]")
            viewer = BrowserViewerServer(client, port=8000)
            viewer_url = viewer.start(open_browser=True)

            console.print("\n[bold green]Viewer Features:[/bold green]")
            console.print(
                "• Default display: 1600×900 (configured via displayLayout callback)"
            )
            console.print("• Size options: 720p, 900p, 1080p, 1440p")
            console.print("• Real-time display updates")
            console.print("• Take/Release control functionality")

            console.print("\n[yellow]Press Ctrl+C to stop[/yellow]")

            # Nova Act agent 초기화 및 브라우저 제어
            with NovaAct(
                cdp_endpoint_url=ws_url,  # Chrome DevTools Protocol endpoint
                cdp_headers=headers,
                preview={"playwright_actuation": True},  # Playwright 기반 브라우저 제어 활성화
                nova_act_api_key=nova_act_key,
                starting_page=starting_page,
            ) as nova_act:
                # 자연어 명령 실행
                result = nova_act.act(prompt)
                console.print(f"\n[bold green]Nova Act Result:[/bold green] {result}")
        
    except Exception as e:
        console.print(f"\n[red]Error: {e}[/red]")
        import traceback
        traceback.print_exc()
    finally:
        console.print("\n\n[yellow]Shutting down...[/yellow]")
        if "client" in locals():
            client.stop()
            console.print("✅ Browser session terminated")
    return result


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--prompt", required=True, help="Browser Search instruction")
    parser.add_argument("--starting-page", required=True, help="Starting URL")
    parser.add_argument("--nova-act-key", required=True, help="Nova Act API key")
    parser.add_argument("--region", default="us-west-2", help="AWS region")
    args = parser.parse_args()

    result = live_view_with_nova_act(
        args.prompt, args.starting_page, args.nova_act_key, args.region
    )

    with open('result.txt', 'w') as f:
        f.write(str(result))

    console.print(f"\n[bold green]Nova Act Result:[/bold green] {result}")

#### 스크립트 실행하기
스크립트를 실행하기 전에 아래에 Nova Act API key를 붙여넣으세요. 

In [ ]:
NOVA_ACT_KEY= ''  ### Nova Act Key를 여기에 붙여넣으세요

In [ ]:
!python live_view_with_nova_act.py --prompt "Search for macbooks and extract the details of the first one" --starting-page "https://www.amazon.com/" --nova-act-key {NOVA_ACT_KEY}

### 내부적으로 무슨 일이 일어났나요? 
* Browser 클라이언트를 인스턴스화하고 세션을 시작했습니다
* 그런 다음 `BrowserViewerServer`를 사용하여 브라우저 세션에 연결하여 로컬에서 세션을 확인했습니다
* 그런 다음 Nova Act Agent를 생성하고 브라우저 세션의 세부 정보를 전달했습니다
* 그런 다음 Nova Act agent에 자연어 지시사항을 전송하고 실시간으로 작업을 확인했습니다

## 브라우저에서 CAPTCHA 처리하기 
다음으로, 브라우저에서 captcha를 처리하는 방법을 살펴보겠습니다. captcha의 목적은 봇이 아닌 사람이 웹사이트와 상호작용하고 있는지 확인하는 것입니다. 따라서 agent가 captcha를 해결하도록 허용하지 않고, 대신 우리가 직접 개입하여 captcha를 해결하고 agent가 계속 진행하도록 합니다. 

새로운 스크립트를 만들어 보겠습니다. Nova Act는 페이지에 captcha가 있는지 확인할 수 있게 해줍니다 - 이 기능을 사용하여 Nova Act가 계속 진행하기 전에 captcha를 처리합니다. 

#### 스크립트가 실행되기 시작하면 브라우저의 로컬 뷰가 표시됩니다. 스크립트가 실행되는 동안 captcha가 표시되면 수동으로 captcha를 해결하세요 - 스크립트는 captcha가 해결될 때까지 대기합니다. 

##### 참고: captcha가 표시되지 않고 스크립트가 완전히 성공적으로 실행되면, captcha가 나타날 때까지 스크립트를 다시 실행해 보세요. 

In [ ]:
%%writefile captcha_with_nova_act.py
from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct, BOOL_SCHEMA, ActAgentError
from rich.console import Console
from rich.panel import Panel
import sys
import json
import time
import argparse
sys.path.append("../interactive_tools")
from browser_viewer import BrowserViewerServer


console = Console()

from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print("using region", region)

def contains_human_validation_error(err):
    """에러 객체에 HumanValidationError가 포함되어 있는지 재귀적으로 확인"""
    if err is None:
        return False

    if isinstance(err, str) and "HumanValidationError" in err:
        return True

    # message 속성이 있으면 재귀 호출
    if hasattr(err, "message"):
        return contains_human_validation_error(err.message)

    if "HumanValidationError" in str(err):
        return True

    return False

def live_view_with_nova_act(steps, starting_page, nova_act_key, region="us-west-2"):
    """Run the browser live viewer with display sizing."""
    console.print(
        Panel(
            "[bold cyan]Browser Live Viewer[/bold cyan]\n\n"
            "This demonstrates:\n"
            "• Live browser viewing with DCV\n"
            "• Configurable display sizes (not limited to 900×800)\n"
            "• Proper display layout callbacks\n\n"
            "[yellow]Note: Requires Amazon DCV SDK files[/yellow]",
            title="Browser Live Viewer",
            border_style="blue",
        )
    )
    result = None

    try:
        # Bedrock AgentCore 브라우저 세션 생성
        with browser_session(region) as client:
            # WebSocket URL과 헤더 생성 (CDP 연결용)
            ws_url, headers = client.generate_ws_headers()

            # 브라우저 실시간 뷰어 서버 시작
            console.print("\n[cyan]Step 3: Starting viewer server...[/cyan]")
            viewer = BrowserViewerServer(client, port=8000)
            viewer_url = viewer.start(open_browser=True)

            console.print("\n[bold green]Viewer Features:[/bold green]")
            console.print(
                "• Default display: 1600×900 (configured via displayLayout callback)"
            )
            console.print("• Size options: 720p, 900p, 1080p, 1440p")
            console.print("• Real-time display updates")
            console.print("• Take/Release control functionality")

            console.print("\n[yellow]Press Ctrl+C to stop[/yellow]")

            # Nova Act agent 초기화 및 브라우저 제어
            with NovaAct(
                cdp_endpoint_url=ws_url,  # Chrome DevTools Protocol endpoint
                cdp_headers=headers,
                preview={"playwright_actuation": True},  # Playwright 기반 브라우저 제어 활성화
                nova_act_api_key=nova_act_key,
                starting_page=starting_page,
            ) as nova_act:
                
                # 각 단계별로 실행 (CAPTCHA 처리 포함)
                for step_index, step in enumerate(steps):
                    max_retries = 3
                    retry_count = 0
                    
                    while retry_count < max_retries:
                        try:
                            print(f"Executing step {step_index + 1}/{len(steps)}: {step}")
                            result = nova_act.act(step)
                            console.print(f"\n[bold green]Step {step_index + 1} Result:[/bold green] {result}")
                            break  # 성공 시 다음 단계로
                            
                        except ActAgentError as err:
                            # CAPTCHA 에러 감지
                            if contains_human_validation_error(err):
                                print("CAPTCHA detected! Please solve it in the browser.")
                                captcha_wait_attempts = 0
                                max_captcha_wait_attempts = 8
                                
                                # CAPTCHA 해결 대기 루프
                                while captcha_wait_attempts < max_captcha_wait_attempts:
                                    try:
                                        time.sleep(10)  # 사용자가 CAPTCHA 해결할 시간 제공
                                        # CAPTCHA 존재 여부 확인
                                        captcha_result = nova_act.act(
                                            "Is there a captcha on the screen?", schema=BOOL_SCHEMA
                                        )
                                        
                                        # CAPTCHA가 해결되었으면 현재 단계 재시도
                                        if captcha_result.matches_schema and not captcha_result.parsed_response:
                                            print("Captcha solved, continuing with current step...")
                                            # retry_count 증가 없이 재시도
                                            break
                                        else:
                                            print(f"Captcha still present. Waiting... (Attempt {captcha_wait_attempts + 1}/{max_captcha_wait_attempts})")
                                            captcha_wait_attempts += 1
                                            
                                    except Exception as captcha_check_err:
                                        print(f"Error checking captcha status: {str(captcha_check_err)}")
                                        captcha_wait_attempts += 1
                                        time.sleep(5)
                                
                                if captcha_wait_attempts >= max_captcha_wait_attempts:
                                    print("Maximum captcha wait attempts reached. Trying to continue anyway.")
                                    retry_count += 1
                                
                            else:
                                print(f"Non-captcha error occurred: {str(err)}")
                                retry_count += 1
                                time.sleep(5)
                                
                        except Exception as general_err:
                            print(f"Unexpected error on step {step_index + 1}: {str(general_err)}")
                            retry_count += 1
                            time.sleep(5)
                            
                    if retry_count >= max_retries:
                        console.print(f"\n[bold red]Failed to complete step {step_index + 1} after {max_retries} attempts.[/bold red]")
                        if step_index < len(steps) - 1:
                            console.print("[yellow]Attempting to continue with next step...[/yellow]")
                
                console.print("\n[bold blue]Task Execution Complete[/bold blue]")
        
    except Exception as e:
        console.print(f"\n[red]Error: {e}[/red]")
        import traceback
        traceback.print_exc()
    finally:
        console.print("\n\n[yellow]Shutting down...[/yellow]")
        if "client" in locals():
            client.stop()
            console.print("✅ Browser session terminated")
    return result


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--steps", required=False, help="JSON array or comma-separated list of steps to execute", 
                        default='["Search for AI news and press enter. If there is alredy AI news typed in search bar, then do not do anything", "Get the first AI news result, open the page and extract the title. Instead, if you see an AI summary, extract the first paragrpah of the summary and return"]')
    parser.add_argument("--starting-page", required=True, help="Starting URL")
    parser.add_argument("--nova-act-key", required=True, help="Nova Act API key")
    parser.add_argument("--region", default="us-west-2", help="AWS region")
    args = parser.parse_args()

    # JSON 배열 또는 쉼표로 구분된 문자열로 steps 파싱
    try:
        # JSON 파싱 시도
        steps = json.loads(args.steps)
    except json.JSONDecodeError:
        # JSON이 아니면 쉼표로 구분된 문자열로 처리
        steps = [step.strip() for step in args.steps.split(',')]

    # steps가 리스트인지 확인
    if not isinstance(steps, list):
        steps = [steps]

    result = live_view_with_nova_act(
        steps, args.starting_page, args.nova_act_key, args.region
    )

In [ ]:
!python captcha_with_nova_act.py  --starting-page "https://www.google.com/" --nova-act-key {NOVA_ACT_KEY}

# 축하합니다!